In [1]:
import numpy as np 
import pandas as pd 

np.random.seed(100)

In [2]:
def Create_Warehouses(NumberOfWarehouses=5, Latitude_Range=[34, 35], Longitude_Range=[-81, -80]):

    Warehouse_df = pd.DataFrame({
        "WarehouseID": pd.Series(dtype=int),
        "WH_Latitude": pd.Series(dtype=float),
        "WH_Longitude": pd.Series(dtype=float),
        "Volume": pd.Series(dtype=int)
    })

    Warehouse_df["WarehouseID"] = np.arange(1, NumberOfWarehouses + 1, 1, dtype=int) * 1000
    Latitude_Min, Latitude_Max = Latitude_Range[0], Latitude_Range[1]
    Longitude_Min, Longitude_Max = Longitude_Range[0], Longitude_Range[1]

    Warehouse_df["WH_Latitude"] = np.random.uniform(Latitude_Min, Latitude_Max, NumberOfWarehouses)
    Warehouse_df["WH_Longitude"] = np.random.uniform(Longitude_Min, Longitude_Max, NumberOfWarehouses)

    Warehouse_df["Volume"] = np.random.randint(2, 8, NumberOfWarehouses) # The volume each WH gets

    return Warehouse_df

wh_df = Create_Warehouses(NumberOfWarehouses=10)
wh_df

,WarehouseID,WH_Latitude,WH_Longitude,Volume
0,1000,34.543405,-80.108678,5
1,2000,34.278369,-80.790798,5
2,3000,34.424518,-80.814672,5
3,4000,34.844776,-80.891623,3
4,5000,34.004719,-80.780303,3
5,6000,34.121569,-80.021376,7
6,7000,34.670749,-80.188317,5
7,8000,34.825853,-80.828059,2
8,9000,34.136707,-80.183775,4
9,10000,34.575093,-80.725926,3


In [41]:
iter_df = pd.DataFrame({"ShipmentDate": pd.date_range(start="2025-01-01", end="2025-08-31").tolist()})
df = pd.DataFrame(columns=["ShipmentID", "ShipmentDate", "WarehouseID", "ItemCount", "TotalWeight", "COGS", "Revenue", "Profit", "ShipmentType"])

for i in range(len(iter_df)):
    ShipmentDate = pd.to_datetime(iter_df.ShipmentDate.iloc[i])

    shipment_types = ["standard", "standard", "standard", "expedited", "expedited", "express", "express", "express", "overnight"]
    shipment_types = [s_type.upper() for s_type in shipment_types]


    for j in range(len(wh_df)):

        join_df = pd.DataFrame()

        WarehouseID = wh_df.WarehouseID.iloc[j]
        Volume = wh_df.Volume.iloc[j]

        cnt_shipments = np.random.randint(low=0, high=Volume)

        join_df["ShipmentID"] = np.random.randint(10000, 99999, size=cnt_shipments)
        join_df["ShipmentDate"] = [ShipmentDate for i in range(cnt_shipments)]
        join_df["WarehouseID"] = [WarehouseID for i in range(cnt_shipments)]
        join_df["ItemCount"] = np.random.randint(100, 1000, size=cnt_shipments)
        join_df["TotalWeight"] = np.random.uniform(1000, 10000, size=cnt_shipments)
        join_df["COGS"] = join_df["TotalWeight"] * np.random.uniform(0, 0.5, size=cnt_shipments)
        join_df["Revenue"] = join_df["TotalWeight"] * np.random.uniform(2, 3, size=cnt_shipments)
        join_df["Profit"] = join_df["Revenue"] - join_df["COGS"]
        join_df["ShipmentType"] = np.random.choice(shipment_types, size=cnt_shipments)

        df = pd.concat([df, join_df])


In [42]:
df = pd.merge(df, wh_df, on="WarehouseID", suffixes=('', '_y'))
df = df.drop("Volume", axis=1)
df

,ShipmentID,ShipmentDate,WarehouseID,ItemCount,TotalWeight,COGS,Revenue,Profit,ShipmentType,WH_Latitude,WH_Longitude
0,93139,2025-01-01 00:00:00,1000,420,3150.878276,1276.782111,8313.686474,7036.904363,EXPRESS,34.543405,-80.108678
1,68036,2025-01-01 00:00:00,1000,432,8267.060368,3122.110472,22629.560222,19507.44975,STANDARD,34.543405,-80.108678
2,89432,2025-01-01 00:00:00,1000,355,8263.377,206.855982,17423.982963,17217.126981,EXPRESS,34.543405,-80.108678
3,43349,2025-01-01 00:00:00,1000,616,9482.709861,3740.622771,22305.298589,18564.675818,EXPEDITED,34.543405,-80.108678
4,82297,2025-01-01 00:00:00,2000,939,3969.197852,215.841926,11414.116205,11198.274279,EXPRESS,34.278369,-80.790798
...,...,...,...,...,...,...,...,...,...,...,...
3970,38593,2025-08-31 00:00:00,6000,797,9719.054813,2242.905026,23346.297423,21103.392397,OVERNIGHT,34.121569,-80.021376
3971,35156,2025-08-31 00:00:00,6000,155,1170.032776,558.934787,3439.960851,2881.026065,EXPEDITED,34.121569,-80.021376
3972,57546,2025-08-31 00:00:00,6000,996,8128.233675,2492.470016,19583.380321,17090.910305,STANDARD,34.121569,-80.021376
3973,50437,2025-08-31 00:00:00,10000,741,1915.314636,100.700804,4367.498188,4266.797385,EXPRESS,34.575093,-80.725926


In [43]:
df.to_excel("Warehouse_Data.xlsx")

In [44]:
group_df = df.copy()

group_df["ShipmentMonth"] = pd.to_datetime(group_df.ShipmentDate).dt.month
group_df["ShipmentMonthName"] = pd.to_datetime(group_df.ShipmentDate).dt.month_name().str[:3]
group_df = group_df.groupby(["ShipmentMonth", "ShipmentMonthName"]).agg({
    "ShipmentID": "count",
    "TotalWeight": "sum",
    "COGS": "sum",
    "Revenue": "sum",
    "Profit": "sum"
})

group_df_shift = group_df.shift(1)

group_df = group_df.reset_index()
group_df_shift = group_df_shift.reset_index()

goals_df = pd.merge(group_df, group_df_shift, on="ShipmentMonth", suffixes=("", "_Goal"))
goals_df = goals_df.drop("ShipmentMonthName_Goal", axis=1)
goals_df

,ShipmentMonth,ShipmentMonthName,ShipmentID,TotalWeight,COGS,Revenue,Profit,ShipmentID_Goal,TotalWeight_Goal,COGS_Goal,Revenue_Goal,Profit_Goal
0,1,Jan,507,2792837.165013,701908.885114,6982556.483854,6280647.598739,NaN,None,None,None,None
1,2,Feb,462,2601291.318499,637058.863623,6482043.934666,5844985.071044,507.0,2792837.165013,701908.885114,6982556.483854,6280647.598739
2,3,Mar,563,3011833.804045,749904.62161,7468064.018869,6718159.397259,462.0,2601291.318499,637058.863623,6482043.934666,5844985.071044
3,4,Apr,486,2655861.341491,652425.13129,6631559.916667,5979134.785377,563.0,3011833.804045,749904.62161,7468064.018869,6718159.397259
4,5,May,496,2665165.693381,701789.758047,6691733.050147,5989943.2921,486.0,2655861.341491,652425.13129,6631559.916667,5979134.785377
5,6,Jun,473,2647079.394493,667569.97752,6615602.931952,5948032.954432,496.0,2665165.693381,701789.758047,6691733.050147,5989943.2921
6,7,Jul,469,2647074.859254,641390.423588,6667248.58809,6025858.164503,473.0,2647079.394493,667569.97752,6615602.931952,5948032.954432
7,8,Aug,519,2841693.401474,684453.622366,7079406.076932,6394952.454566,469.0,2647074.859254,641390.423588,6667248.58809,6025858.164503


In [45]:
goals_df.to_excel("Warehouse_Data_Goals.xlsx")